In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np

In [14]:
class Threemm(nn.Module):
    def __init__(self):
        super(Threemm, self).__init__()

    def forward(self, X):
        n = 10240 # X.shape[0] // 4
        B = X[0:n, :] #X[0:n,:]
        C = X[n:2*n,:] #X[n:2*n,:]
        E = X[2*n:3*n,:] #X[2*n:3*n,:]
        F = X[3*n:4*n,:] #X[3*n:4*n,:]
        # A=BC, D=EF, G=AD
        A = torch.matmul(B, C)
        D = torch.matmul(E, F)
        G = torch.matmul(A, D)
        return G

threemm = Threemm()
# threemm = threemm.cuda()

In [15]:
# test 3mm
N = 10240
X = torch.randn((4*N, N))

In [16]:
output_threemm  = threemm(X)
output_threemm.shape

torch.Size([10240, 10240])

In [17]:
torch.onnx.export(threemm, X, 'threemm.onnx')

In [18]:
tensor = np.random.rand(40960, 10240)
np.save("tensor-4096-1024", np.float32(tensor))

In [ ]:
class Nmm(nn.Module):
    def __init__(self):
        super(Nmm, self).__init__()

    def forward(self, X):
        n = 1024 # X.shape[0] // 4
        B1 = X[0:n, :] #X[0:n,:]
        B2 = X[n:2*n,:] #X[n:2*n,:]
        B3 = X[2*n:3*n,:] #X[2*n:3*n,:]
        B4 = X[3*n:4*n,:] #X[3*n:4*n,:]
        B5 = X[4*n:5*n,:] 
        B6 = X[5*n:6*n,:]

        # A=BC, D=EF, G=AD
        A1 = torch.matmul(B1, B2)
        A2 = torch.matmul(B3, B4)
        A3 = torch.matmul(B5, B6)

        C1 = torch.matmul(A1, A2)
        C2 = torch.matmul(A2, A3)

        G = torch.matmul(C1, C2)
        return G

nmm = Nmm()
# threemm = threemm.cuda()

In [ ]:
# test 3mm
N = 1024
X = torch.randn((6*N, N))

In [ ]:
output_nmm  = nmm(X)
output_nmm.shape

In [ ]:
torch.onnx.export(nmm, X, 'nmm.onnx')